In [ ]:
# default_exp features
# default_cls_lvl 2

In [ ]:
%load_ext autotime

# Feature Engineering und Target Variablen
> In diesem Modul werden zusätzliche Features berechnet und die Zielvariablen sowie Test und Train split definiert.

Laut [Kaggle Data Description](https://www.kaggle.com/c/santander-product-recommendation/data) haben wir 1.5 Jahre an Daten. Das besagte Ziel ist es die zusätzlich erworbenen Produkte vorherzusagen für die Periode 2016-05-28. Dies ist eines der wichtigsten Schritte im ganzen ML Prozess. Für diesen PoC halten wir das ganze Recht einfach. In eine echten Projekt würden wir zusätzlich folgende Schritte machen:

 - entwicklen von zusätzlichen Featueres wie z.B. relatives Einkommen zur Altersgruppe und Lokation
 - berechnen von Differenzen zum Vormonat bzw. Vormonaten
 - mehrere Datasets definieren für ein Ensemble von Modellen.
 
 
Das Ziel dieser Kaggle Challenge ist es die neuen Produkte für die Periode 2016-06-28 vorherzusagen. Dazu haben sie das Datenset in ein Testset (Grunddaten von der Periode 2016-06-28) und Trainingset (Daten von 2015-01-28 bis 2016-05-28) aufgeteilt. Leider kennen wir die wahren Werte von dem Testset der Periode 2016-06-28 nicht, weshalb wir diesen Datenpunkt ignorieren werden und Train und Testset wie folgt aufteilen werden. 
![image.png](docs/images/train_test.png)

In [ ]:
#export
import pandas as pd
import numpy as np
from fastscript import *

time: 111 ms


In [ ]:
#export
def load_data(path='data/interim/02_train.csv'):
    """load data"""
    return pd.read_csv(path)

time: 329 µs


In [ ]:
data = load_data()

time: 30.9 s


In [ ]:
data.head(10)

,id,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,...,month_int,fecha_alta_month,fecha_alta_year,fecha_alta_day,fecha_alta_month_int,fecha_alta_day_int,ult_fec_cli_1t_month,ult_fec_cli_1t_year,ult_fec_cli_1t_day,ult_fec_cli_1t_month_int
0,1018027,0,0,1,35,0,38,1,1,0,...,1,5,17,30,209,6385,1,5,1,61
1,1312827,0,0,0,22,0,10,1,1,1,...,1,9,19,22,237,7227,1,5,1,61
2,239244,0,0,1,41,0,171,1,1,0,...,1,4,6,10,76,2320,1,5,1,61
3,1096938,0,0,1,23,0,32,1,1,1,...,1,11,17,7,215,6542,1,5,1,61
4,259247,0,0,1,56,0,168,1,1,0,...,1,4,9,12,112,3417,1,5,1,61
5,1096948,0,0,0,23,0,32,1,1,1,...,1,11,17,7,215,6542,1,5,1,61
6,951377,0,0,1,37,0,46,1,1,1,...,1,9,16,29,201,6139,1,5,1,61
7,428293,0,0,0,42,0,144,1,1,1,...,1,7,8,21,103,3151,1,5,1,61
8,259245,0,0,0,54,0,168,1,1,0,...,1,7,6,4,79,2404,1,5,1,61
9,1154223,0,0,0,49,0,23,1,1,0,...,1,8,18,2,224,6812,1,5,1,61


time: 14 ms


In [ ]:
#export
target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
               'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
               'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
               'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
               'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
               'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
               'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

target_cols.remove('ind_ahor_fin_ult1') #wenig häufig
target_cols.remove('ind_aval_fin_ult1') #wenig häufig


feature_cols = ['ind_empleado', 'sexo', 'age', 'renta', 'ind_nuevo', 
                'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 
                'conyuemp', 'indfall', 'tipodom', 'ind_actividad_cliente', 
                'segmento', 'antiguedad', 'pais_residencia', 'canal_entrada']

time: 996 µs


In [ ]:
data[target_cols].sum()

ind_cco_fin_ult1     7110470
ind_cder_fin_ult1       4544
ind_cno_fin_ult1      953036
ind_ctju_fin_ult1     111560
ind_ctma_fin_ult1     100151
ind_ctop_fin_ult1    1494665
ind_ctpp_fin_ult1     508713
ind_deco_fin_ult1      18439
ind_deme_fin_ult1      19128
ind_dela_fin_ult1     505756
ind_ecue_fin_ult1     971571
ind_fond_fin_ult1     215673
ind_hip_fin_ult1       70728
ind_plan_fin_ult1     109810
ind_pres_fin_ult1      28475
ind_reca_fin_ult1     623697
ind_tjcr_fin_ult1     526745
ind_valo_fin_ult1     296541
ind_viv_fin_ult1       46521
ind_nomina_ult1       647960
ind_nom_pens_ult1     704080
ind_recibo_ult1      1492387
dtype: int64

time: 481 ms


## calculate target vars

In [ ]:
#export
def calculate_targets(df:pd.DataFrame, 
                      feature_cols:list=feature_cols, 
                      target_products:list=target_cols,
                      remove_wrong_shift_entries=True,
                      shift_periods=1):
    """add the shifted product values and calculate target variables"""
    
    df.sort_values(by = ['id', 'month_int'], inplace=True) #sort by id then by month_int
    df['id_shift'] = df['id'].shift(shift_periods).fillna(0).astype(np.int32)
    
    idx_to_remove = ((df['id'] - df['id_shift']) != 0) #store index unwanted entries
    
    #add shifted target colums
    for col in target_products:    
        name = col + '_s'
        df[name] = df[col].shift(shift_periods).fillna(0).astype(np.int8)        
        df.loc[idx_to_remove, name] = 0 #set to 0 so that the difference works out
        
    # set 1 only for added products not for existing products
    for col in target_products:
        df[col] = (df[col] - df[col + '_s']).astype(np.int8)
        df[col] = (df[col] > 0).astype(np.int8)
        
    if remove_wrong_shift_entries:
        df = df[idx_to_remove == False] #remove illogical results    
    
    return df


time: 1.24 ms


In [ ]:
df1 = calculate_targets(data)
df_s_12 = calculate_targets(data, shift_periods = 12)
print(len(df1))
print(len(df_s_12))
assert len(df1) > len(df_s_12)
assert min(df_s_12.month_int.unique()) == 13 #first month 1 + 12 shift
assert min(calculate_targets(data, shift_periods = 6).month_int.unique()) == 7
assert 'ind_ctma_fin_ult1_s' in df1.columns #shifted columns


10143523
2569018
time: 1min 13s


In [ ]:
#export 
def remove_rows_without_product(df:pd.DataFrame, target_products:list=target_cols):
    """removes all rows from a train set without new products"""
    return df[df[target_products].sum(axis=1) != 0] #entfernen von Zeilen ohne neue Produkte

time: 427 µs


In [ ]:
df2 = remove_rows_without_product(df1)
assert len(df2) < len(df1)

time: 511 ms


In [ ]:
#export
def calculate_target_as_one_column(df:pd.DataFrame, feature_cols:list, target_cols:list):
    """create a row for every new porduct and give the product name as target column, this is done for the train set"""
    x = df[target_cols]
    x = x[x==1].stack().reset_index().drop(0,1)
    df = pd.merge(df, x, left_on=df.index, right_on='level_0')
    df.rename(columns={'level_1': "y"}, inplace=True)
    
    keep_cols = feature_cols.copy()
    keep_cols += [ col for col in df if col[-2:] == '_s'] # keep also shifted columns
    keep_cols.append('month_int')
    keep_cols.append('id') #keep id
    keep_cols.append('y') #keep target var  
    
    return df[keep_cols]

time: 1.24 ms


In [ ]:
df3 = calculate_target_as_one_column(df2, feature_cols, target_cols)
assert 'y' in df3.columns
assert len(df3) > len(df2) #additional rows as there are more than one product per month

time: 384 ms


## train_test_split

In [ ]:
#export 
def get_last_month_test_set(df:pd.DataFrame, feature_cols=feature_cols, target_cols=target_cols):
    """calculates test set for all new products in the last month"""
    df = df.loc[df.month_int >= 16,].copy()
    df = calculate_targets(df, feature_cols, target_cols, shift_periods=1, remove_wrong_shift_entries=False)
    df = df.loc[df.month_int == 17]
    return df

time: 580 µs


In [ ]:
df4_test = get_last_month_test_set(data)
assert df4_test.month_int.unique()[0] == 17
assert data[data.month_int == 17]['month_int'].count() == len(df4_test)

time: 1.36 s


In [ ]:
#export
def get_train_set(df:pd.DataFrame, feature_cols=feature_cols, target_cols=target_cols, product_shift=1):
    """retunrs the trainset. Take care that you don't include the test data to this function"""
    train = calculate_targets(df.loc[df.month_int <= 16,].copy(), 
                              feature_cols, target_cols, 
                              shift_periods=product_shift)
    train = remove_rows_without_product(train, target_cols)
    train = calculate_target_as_one_column(train, feature_cols, target_cols)
    return train

time: 607 µs


In [ ]:
df5_train = get_train_set(data)
assert 'y' in df5_train.columns
assert 'id' in df5_train.columns
assert df5_train.month_int.max() == 16

time: 8.19 s


In [ ]:
df5_train

,ind_empleado,sexo,age,renta,ind_nuevo,indrel,indrel_1mes,tiprel_1mes,indresi,indext,...,ind_reca_fin_ult1_s,ind_tjcr_fin_ult1_s,ind_valo_fin_ult1_s,ind_viv_fin_ult1_s,ind_nomina_ult1_s,ind_nom_pens_ult1_s,ind_recibo_ult1_s,month_int,id,y
0,3,0,56,326124,0,1,1,0,1,0,...,0,0,0,0,0,0,0,5,15889,ind_tjcr_fin_ult1
1,3,0,56,326124,0,1,1,0,1,0,...,0,0,0,0,0,0,0,12,15889,ind_tjcr_fin_ult1
2,3,0,56,326124,0,1,1,0,1,0,...,0,0,0,0,0,0,0,15,15889,ind_tjcr_fin_ult1
3,3,1,61,430477,0,1,1,0,1,0,...,0,0,0,0,0,0,0,6,15892,ind_cco_fin_ult1
4,3,1,61,430477,0,1,1,0,1,0,...,0,0,0,0,0,0,0,9,15892,ind_dela_fin_ult1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369626,0,0,45,75445,1,1,1,0,1,0,...,0,0,0,0,0,0,0,11,1454615,ind_cno_fin_ult1
369627,0,0,45,75445,1,1,1,0,1,0,...,0,0,0,0,0,0,0,12,1454615,ind_nomina_ult1
369628,0,0,45,75445,1,1,1,0,1,0,...,0,0,0,0,0,0,0,12,1454615,ind_nom_pens_ult1
369629,0,0,45,75445,1,1,1,0,1,0,...,0,0,0,0,0,0,0,14,1454615,ind_nomina_ult1


time: 40.5 ms


In [ ]:
#export
@call_parse
def calculate_main(source:Param("source csv file", str)='data/interim/02_train.csv',
                   dest_train:Param("destination train csv file", str)='data/interim/03_train.csv',
                   dest_test:Param("destination test csv file", str)='data/interim/03_test.csv',
                   shift_periods:Param("how many periods to shift target_vars", str) = 1):
    """calculate target variables and delayed product features"""
    
    data = load_data(source) 
    train = get_train_set(data, product_shift=shift_periods)
    test = get_last_month_test_set(data)
    
    train.to_csv(dest_train, index=False)
    test.to_csv(dest_test, index=False)
    return (train, test)
    

time: 62.8 ms


In [ ]:
#slow
train, test = calculate_main()
train12, test12 = calculate_main(
    dest_train='data/interim/03_train_shift12.csv', 
    dest_test='data/interim/03_test_shift12.csv', 
    shift_periods=12)
train6, test6 = calculate_main(
    dest_train='data/interim/03_train_shift6.csv', 
    dest_test='data/interim/03_test_shift6.csv', 
    shift_periods=6)

time: 3min 54s


In [ ]:
#slow
pd.set_option('display.max_columns', None)
train

,ind_empleado,sexo,age,renta,ind_nuevo,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,indfall,tipodom,ind_actividad_cliente,segmento,antiguedad,pais_residencia,canal_entrada,ind_cco_fin_ult1_s,ind_cder_fin_ult1_s,ind_cno_fin_ult1_s,ind_ctju_fin_ult1_s,ind_ctma_fin_ult1_s,ind_ctop_fin_ult1_s,ind_ctpp_fin_ult1_s,ind_deco_fin_ult1_s,ind_deme_fin_ult1_s,ind_dela_fin_ult1_s,ind_ecue_fin_ult1_s,ind_fond_fin_ult1_s,ind_hip_fin_ult1_s,ind_plan_fin_ult1_s,ind_pres_fin_ult1_s,ind_reca_fin_ult1_s,ind_tjcr_fin_ult1_s,ind_valo_fin_ult1_s,ind_viv_fin_ult1_s,ind_nomina_ult1_s,ind_nom_pens_ult1_s,ind_recibo_ult1_s,month_int,id,y
0,3,0,56,326124,0,1,1,0,1,0,0,0,1,1,1,245,0,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5,15889,ind_tjcr_fin_ult1
1,3,0,56,326124,0,1,1,0,1,0,0,0,1,1,1,250,0,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,12,15889,ind_tjcr_fin_ult1
2,3,0,56,326124,0,1,1,0,1,0,0,0,1,1,1,253,0,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,15,15889,ind_tjcr_fin_ult1
3,3,1,61,430477,0,1,1,0,1,0,0,0,1,1,1,246,0,5,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,6,15892,ind_cco_fin_ult1
4,3,1,61,430477,0,1,1,0,1,0,0,0,1,1,1,248,0,5,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,1,9,15892,ind_dela_fin_ult1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421949,0,0,45,75445,1,1,1,0,1,0,-1,0,1,1,2,2,0,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1454615,ind_cno_fin_ult1
421950,0,0,45,75445,1,1,1,0,1,0,-1,0,1,1,2,3,0,21,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,1454615,ind_nomina_ult1
421951,0,0,45,75445,1,1,1,0,1,0,-1,0,1,1,2,3,0,21,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,1454615,ind_nom_pens_ult1
421952,0,0,45,75445,1,1,1,0,1,0,-1,0,1,1,2,5,0,21,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,1454615,ind_nomina_ult1


time: 52.7 ms


In [ ]:
#slow
test

,id,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_dato_month,fecha_dato_year,month_int,fecha_alta_month,fecha_alta_year,fecha_alta_day,fecha_alta_month_int,fecha_alta_day_int,ult_fec_cli_1t_month,ult_fec_cli_1t_year,ult_fec_cli_1t_day,ult_fec_cli_1t_month_int,id_shift,ind_cco_fin_ult1_s,ind_cder_fin_ult1_s,ind_cno_fin_ult1_s,ind_ctju_fin_ult1_s,ind_ctma_fin_ult1_s,ind_ctop_fin_ult1_s,ind_ctpp_fin_ult1_s,ind_deco_fin_ult1_s,ind_deme_fin_ult1_s,ind_dela_fin_ult1_s,ind_ecue_fin_ult1_s,ind_fond_fin_ult1_s,ind_hip_fin_ult1_s,ind_plan_fin_ult1_s,ind_pres_fin_ult1_s,ind_reca_fin_ult1_s,ind_tjcr_fin_ult1_s,ind_valo_fin_ult1_s,ind_viv_fin_ult1_s,ind_nomina_ult1_s,ind_nom_pens_ult1_s,ind_recibo_ult1_s
10649083,15889,3,0,0,56,0,255,1,1,0,1,0,0,5,0,1,28,1,326124,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5,1,17,1,0,16,1,46,1,5,1,61,15889,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10649181,15890,1,0,0,63,0,256,1,1,0,1,0,0,5,0,1,28,1,71461,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,1,0,16,1,46,1,5,1,61,15890,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1
10649180,15892,3,0,1,62,0,256,1,1,0,1,0,0,5,0,1,28,1,430477,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,1,0,16,1,46,1,5,1,61,15892,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1
10649179,15893,0,0,0,63,0,256,1,1,0,1,0,0,5,0,1,28,1,430477,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,10,2,3,34,1033,1,5,1,61,15893,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
10649167,15894,1,0,0,60,0,256,1,1,0,1,0,0,5,0,1,28,1,281757,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,1,0,16,1,46,1,5,1,61,15894,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10824630,1454615,0,0,0,46,0,8,1,1,0,1,0,-1,21,0,1,8,1,75445,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,9,20,18,249,7588,1,5,1,61,1454615,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10540503,1454616,0,0,1,21,0,8,1,1,0,1,0,-1,157,0,1,9,1,132889,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,9,20,18,249,7588,1,5,1,61,1454616,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10607387,1454617,0,0,1,21,0,8,1,1,0,1,0,-1,157,0,1,11,1,58476,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,9,20,18,249,7588,1,5,1,61,1454617,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10205190,1454618,0,0,0,20,0,8,1,1,1,1,0,-1,157,0,1,28,0,75904,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,17,9,20,18,249,7588,1,5,1,61,1454618,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


time: 46.4 ms


In [ ]:
print(train.month_int.min())
print(train6.month_int.min())
print(train12.month_int.min())
print(len(train))
print(len(train6))
print(len(train12))

2
7
13
421954
642460
341934
time: 5.5 ms


## Export

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_data_preprocess.ipynb.
Converted 02_data_Cleaning.ipynb.
Converted 03_features.ipynb.
Converted 04_base_model.ipynb.
Converted 05_xgboost_simple_ensemble.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 06_Distributed_ML.ipynb.
Converted index.ipynb.
time: 175 ms


In [ ]:
len(train)

421954

time: 1.25 ms
